In [1]:
import math 
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import sklearn.datasets as skdatasets
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from Classifier.Model import SingleLaneModel, TwoLaneModel

In [3]:
data_set_path = "/home/thrasolt/git/home_project_students/data/" + "data.pickle"
with open(data_set_path, 'rb') as file_handle:
    dataset = pickle.load(file_handle)

In [4]:
data = dataset[:, :906]
labels = dataset[:, 906]

train_set = dataset[0:320,:]
val_set = dataset[320:,:]

In [5]:
LEARNING_RATE = 0.01
BATCH_SIZE = 25
EPOCHS = 30

In [6]:
in_dim = 906
out_dim=2
inner_layers=[64, 32]

In [7]:
text_dim = 900
text_layers = [256, 64]
emotions_dim= 6
emotions_layers= [16, 16]
combination_dim = 16
combination_layers= [32, 16]

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
dataloader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [10]:
model = TwoLaneModel(text_dim, text_layers, emotions_dim, emotions_layers, combination_dim, combination_layers, out_dim)
#model = SingleLaneModel(in_dim =in_dim, out_dim=out_dim, inner_layers=inner_layers)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [11]:
for epoch in range(EPOCHS):
        loss_avg = 0
        for batch in dataloader:
            
            batch_data = batch[:, :in_dim]
            batch_labels = batch[:, in_dim].long()
            outputs = model(batch_data)

            loss = criterion(outputs, batch_labels)
            loss_avg += loss

            loss.backward()
            optimizer.step()
            model.zero_grad()
        print(f"Epoch {epoch}, Loss: {loss_avg/train_set.shape[0]}")
        dataloader = DataLoader(dataset=train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    

Epoch 0, Loss: 0.028288651257753372
Epoch 1, Loss: 0.027991095557808876
Epoch 2, Loss: 0.027590269222855568
Epoch 3, Loss: 0.027462372556328773
Epoch 4, Loss: 0.027085915207862854
Epoch 5, Loss: 0.027124006301164627
Epoch 6, Loss: 0.02674219012260437
Epoch 7, Loss: 0.026624437421560287
Epoch 8, Loss: 0.026410117745399475
Epoch 9, Loss: 0.02543187513947487
Epoch 10, Loss: 0.025646742433309555
Epoch 11, Loss: 0.025399276986718178
Epoch 12, Loss: 0.022762643173336983
Epoch 13, Loss: 0.024946652352809906
Epoch 14, Loss: 0.021944746375083923
Epoch 15, Loss: 0.021196875721216202
Epoch 16, Loss: 0.019932478666305542
Epoch 17, Loss: 0.021506864577531815
Epoch 18, Loss: 0.02190008945763111
Epoch 19, Loss: 0.018932323902845383
Epoch 20, Loss: 0.020827658474445343
Epoch 21, Loss: 0.01872747763991356
Epoch 22, Loss: 0.017254341393709183
Epoch 23, Loss: 0.016794096678495407
Epoch 24, Loss: 0.016178160905838013
Epoch 25, Loss: 0.015860367566347122
Epoch 26, Loss: 0.016613420099020004
Epoch 27, Loss:

In [12]:
train_data = train_set[:, :in_dim]
train_labels = train_set[:, in_dim]

np.sum(np.argmax(model(torch.tensor(train_data)).detach().numpy(), axis=1)==train_labels)/train_set.shape[0]

0.775

In [13]:
val_data = torch.tensor(val_set[:, :in_dim]).float()
val_labels = torch.tensor(val_set[:, in_dim]).long()

In [14]:
predictions = np.argmax(model(val_data).detach().numpy(), axis=1)

In [15]:
np.sum(predictions == val_labels.detach().numpy())/len(val_labels)

0.625

In [16]:
model_path = "/home/thrasolt/git/home_project_students/models/" + "final_model_1.pytorch"
torch.save(model.state_dict(), model_path)